In [1]:
import tensorflow as tf
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
import json

In [ ]:
from huggingface_hub import login

# Replace with your token from https://huggingface.co/settings/tokens
hf_token = ""
login(token=hf_token)
from huggingface_hub import whoami
print(whoami())  # Should print your account details

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'type': 'user', 'id': '67a59e744e21c6b0419dc7eb', 'name': 'vishnuap25', 'fullname': 'VISHNU AP', 'email': 'vishnuap25@gmail.com', 'emailVerified': True, 'canPay': False, 'periodEnd': None, 'isPro': False, 'avatarUrl': '/avatars/da12fde9980f200f679419aea86b6a72.svg', 'orgs': [], 'auth': {'type': 'access_token', 'accessToken': {'displayName': 'Colab', 'role': 'read', 'createdAt': '2025-02-27T13:31:59.402Z'}}}


In [3]:
model_name = "TinyLlama/TinyLlama_v1.1"
# Load tokenizer & model
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/560 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [4]:
with open('/content/valid.jsonl','r') as file:
  conversations_raw = file.readlines()
conversation_processed = [json.loads(conversation) for conversation in conversations_raw]

with open('/content/test.jsonl','r') as file:
  conversations_eval_raw = file.readlines()
conversation_eval_processed = [json.loads(conversation) for conversation in conversations_raw]

In [5]:
conversation_processed[0]

{'input': 'Where To Get Homeowners Insurance? ',
 'output': 'Homeowners insurance is available through the internet , through direct writers who use call centers or through agents . The difference is the degree of communication you want to have . The agent will be the most personal while the internet will be the least . My suggestion is to ask your colleagues , friends and neighbors and find out who they trust . In the end a trusted counsellor may be the most important part of the homeowners insurance decision . '}

In [6]:
conversation_eval_processed[0]

{'input': 'Where To Get Homeowners Insurance? ',
 'output': 'Homeowners insurance is available through the internet , through direct writers who use call centers or through agents . The difference is the degree of communication you want to have . The agent will be the most personal while the internet will be the least . My suggestion is to ask your colleagues , friends and neighbors and find out who they trust . In the end a trusted counsellor may be the most important part of the homeowners insurance decision . '}

In [7]:
train = []
for conversation in conversation_processed:
  QA = f"Instruction: {conversation['input']} Output: {conversation['output']}"
  tokens = tokenizer(QA, truncation=True, padding="max_length", max_length=1024)
  tokens["labels"]=tokens["input_ids"].copy()
  train.append(tokens)


In [8]:
eval = []
for conversation in conversation_eval_processed:
  QA = f"Instruction: {conversation['input']} Output: {conversation['output']}"
  tokens = tokenizer(QA, truncation=True, padding="max_length", max_length=1024)
  tokens["labels"]=tokens["input_ids"].copy()
  eval.append(tokens)


In [9]:
train[0:3]

[{'input_ids': [1, 2799, 4080, 29901, 6804, 1763, 3617, 8778, 776, 414, 512, 7610, 749, 29973, 29871, 10604, 29901, 8778, 776, 414, 1663, 18541, 338, 3625, 1549, 278, 8986, 1919, 1549, 1513, 23550, 1058, 671, 1246, 1644, 414, 470, 1549, 19518, 869, 450, 4328, 338, 278, 7426, 310, 12084, 366, 864, 304, 505, 869, 450, 10823, 674, 367, 278, 1556, 7333, 1550, 278, 8986, 674, 367, 278, 3203, 869, 1619, 8998, 338, 304, 2244, 596, 23056, 21628, 1919, 7875, 322, 22092, 943, 322, 1284, 714, 1058, 896, 9311, 869, 512, 278, 1095, 263, 9311, 287, 2613, 29879, 514, 272, 1122, 367, 278, 1556, 4100, 760, 310, 278, 3271, 776, 414, 1663, 18541, 10608, 869, 29871, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [10]:
eval[0:3]

[{'input_ids': [1, 2799, 4080, 29901, 6804, 1763, 3617, 8778, 776, 414, 512, 7610, 749, 29973, 29871, 10604, 29901, 8778, 776, 414, 1663, 18541, 338, 3625, 1549, 278, 8986, 1919, 1549, 1513, 23550, 1058, 671, 1246, 1644, 414, 470, 1549, 19518, 869, 450, 4328, 338, 278, 7426, 310, 12084, 366, 864, 304, 505, 869, 450, 10823, 674, 367, 278, 1556, 7333, 1550, 278, 8986, 674, 367, 278, 3203, 869, 1619, 8998, 338, 304, 2244, 596, 23056, 21628, 1919, 7875, 322, 22092, 943, 322, 1284, 714, 1058, 896, 9311, 869, 512, 278, 1095, 263, 9311, 287, 2613, 29879, 514, 272, 1122, 367, 278, 1556, 4100, 760, 310, 278, 3271, 776, 414, 1663, 18541, 10608, 869, 29871, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 

In [11]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs"
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [12]:
# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train,
    eval_dataset=eval
)


In [13]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: vishnu-ai-works (vishnu-ai-works-none) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.312200,0.193627
2,0.169400,0.080991
3,0.062400,0.032426


TrainOutput(global_step=5031, training_loss=0.19788374153953073, metrics={'train_runtime': 17598.119, 'train_samples_per_second': 0.572, 'train_steps_per_second': 0.286, 'total_flos': 6.395451560637235e+16, 'train_loss': 0.19788374153953073, 'epoch': 3.0})

In [14]:
model.save_pretrained("./sft_llm_model")
tokenizer.save_pretrained("./sft_llm_model")

('./sft_llm_model/tokenizer_config.json',
 './sft_llm_model/special_tokens_map.json',
 './sft_llm_model/tokenizer.model',
 './sft_llm_model/added_tokens.json',
 './sft_llm_model/tokenizer.json')

In [123]:
def process_text(text, words_per_line=5, keyword="Output:"):
    parts = text.split(keyword, 1)
    if len(parts) > 1:
      words = parts[1].strip()  # Take everything after the keyword
    else:
      words = None  # Keyword not found
    return (words)

def add_newline_after_n_words(text, n=10):
    words = text.split()  # Split text into words
    result = []

    for i, word in enumerate(words, 1):
        result.append(word)
        if i % n == 0:  # Insert newline after every 'n' words
            result.append("\n")

    return " ".join(result)


In [127]:
import torch
# Define input prompt
prompt = """Act like a insuarance service desk agent and answer the question.
          What Happens When My Term Life Insurance Ends?"""

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

# Generate response
with torch.no_grad():
    outputs = model.generate(**inputs, max_new_tokens=300, do_sample=True, temperature=0.7, top_p=0.6,
                            num_return_sequences=3,eos_token_id=tokenizer.eos_token_id)

# Decode all responses and combine
answers = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
answers = map(process_text, answers)
final = " ".join(list(answers))

print(add_newline_after_n_words(final, 15))

If your Term life insurance policy ends , you will have to re-apply for a 
 new policy . The good news is you can usually get a new policy at 
 your current age . The bad news is you will have to pay the current 
 premium , plus an additional premium based on your new age . This can be 
 quite expensive if you are not insurable at the new age . When your Term 
 life insurance policy ends depends on the term of the policy . If the policy 
 is for 10 years , or 15 , 20 , 25 , or 30 years 
 , most people simply renew their policy for another 10 , 15 , 20 , 
 or 30 years , depending on how long they want the coverage . The premium 
 for the new policy will be higher , however , since the insured has reached 
 the age at which most people are starting to become eligible for long term care 
 benefits . The cost of the insurance is usually lower when the policy is for 
 a shorter period of time . When your Term life insurance policy ends , you 
 no longer have a need to maintain coverage , as